In [2]:
import sagemaker
from sagemaker.pytorch import PyTorch
import pickle
import os
import boto3
import sagemaker
import torch
import argparse
import pickle
# import mlflow
import numpy as np
import json

import torch.nn as nn
import torch.optim as optim

from PIL import Image
from torchvision import models
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader

# # SageMaker Session
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [3]:
s3 = boto3.client('s3')

BUCKET_NAME = 'wildfires'

s3.upload_file(
    f'deployment_step.py',
    BUCKET_NAME,
    f"scripts/deployment.py",
)

In [6]:
import sagemaker
import boto3
from sagemaker.processing import ScriptProcessor
from sagemaker import get_execution_role
from sagemaker import image_uris

role = get_execution_role()
sm_session = sagemaker.Session()

boto_session = boto3.Session()
region = boto_session.region_name

constants = {
    "bucket_name": "wildfires",
    # "sklearn_image_uri_version": "1.2-1",
    "region": boto_session.region_name
}

script_path = sm_session.upload_data(
    path='deployment_step.py',
    bucket=constants["bucket_name"],
    key_prefix="scripts"
)

# test_metadata_path = 's3://wildfires/sagemaker/fire-image-classification'

# best_model_path = 's3://fire-project-hs/34/36ba5f52b9944395ab2ccf94aed56e5a/artifacts/models'

# model_package_arn = 'arn:aws:sagemaker:eu-central-1:567821811420:model-package/first-fire-mlflow-ee0049/1'

# sklearn_image_uri = image_uris.retrieve(
#         framework="sklearn",
#         region=constants["region"],
#         version=constants["sklearn_image_uri_version"],
#     )

script_processor = ScriptProcessor(
    role=role,
    image_uri='763104351884.dkr.ecr.eu-central-1.amazonaws.com/pytorch-inference:2.3.0-gpu-py311-cu121-ubuntu20.04-ec2',
    command=['python3'],
    instance_count=1,
    instance_type='ml.m5.medium'
)

script_processor.run(
    job_name='wildfire-deployment-job-t02',
    code=script_path,
    arguments=[
        # '--test_metadata_bucket', constants["bucket_name"],
        # '--best_model_bucket', constants["bucket_name"],
        # '--test_metadata_prefix', 'sagemaker/fire-image-classification',
        # '--best_model_prefix', 'models',
        # '--test_metadata_file', 'test.pkl',
        # '--best_model_file', 'model_resnet18.tar.gz',
        # '--result_bucket', constants["bucket_name"],
        # '--result_prefix', 'evaluation/dummy',
        # '--result_file', 'eva-result-',
        # '--data_dir', 's3://wildfires/sagemaker/fire-image-classification',
        # '--model_package_arn', model_package_arn,
        '--region', constants["region"]
    ]
)

INFO:sagemaker:Creating processing-job with name wildfire-deployment-job-t02


ClientError: An error occurred (ValidationException) when calling the CreateProcessingJob operation: 1 validation error detected: Value 'ml.m5.medium' at 'processingResources.clusterConfig.instanceType' failed to satisfy constraint: Member must satisfy enum value set: [ml.r5d.12xlarge, ml.r5.12xlarge, ml.m5.4xlarge, ml.p2.xlarge, ml.m4.16xlarge, ml.r5d.24xlarge, ml.r5.24xlarge, ml.t3.xlarge, ml.r5d.16xlarge, ml.g5.2xlarge, ml.r5.16xlarge, ml.m5.large, ml.p3.16xlarge, ml.p2.16xlarge, ml.g5.4xlarge, ml.c4.2xlarge, ml.c5.2xlarge, ml.c4.4xlarge, ml.g5.8xlarge, ml.c5.4xlarge, ml.g4dn.xlarge, ml.g4dn.12xlarge, ml.g4dn.2xlarge, ml.c4.8xlarge, ml.g4dn.4xlarge, ml.c5.9xlarge, ml.g4dn.16xlarge, ml.c5.xlarge, ml.c4.xlarge, ml.g4dn.8xlarge, ml.g5.xlarge, ml.g5.12xlarge, ml.t3.2xlarge, ml.t3.medium, ml.g5.24xlarge, ml.c5.18xlarge, ml.g5.48xlarge, ml.r5d.2xlarge, ml.g5.16xlarge, ml.r5.2xlarge, ml.p3.2xlarge, ml.m5.xlarge, ml.m4.10xlarge, ml.r5d.4xlarge, ml.r5.4xlarge, ml.m5.12xlarge, ml.m4.xlarge, m